<a href="https://colab.research.google.com/github/yanniedog/autocrew/blob/main/run_ollama_remotely_via_google_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# filename: run_ollama_remotely_via_google_collab.ipynb

# Install pyngrok
!pip install pyngrok

from pyngrok import ngrok
from google.colab import userdata
import subprocess

try:
    # Terminate any existing ngrok sessions
    ngrok.kill()

    # Retrieve your ngrok auth token from the Colab secret storage
    auth_token = userdata.get('authtoken')
    if not auth_token:
        raise ValueError("Ngrok auth token not found in Colab secrets.")

    # Set the ngrok authentication token programmatically
    ngrok.set_auth_token(auth_token)

    # Check if Ollama is installed; if not, install it
    try:
        ollama_installed = subprocess.run(["ollama", "--version"], capture_output=True, text=True)
    except FileNotFoundError:
        print("Ollama is not installed. Installing Ollama...")

        # Install Ollama using the provided command
        install_ollama = subprocess.run("curl https://ollama.ai/install.sh | sh", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Check if Ollama installation was successful
        ollama_installed = subprocess.run(["ollama", "--version"], capture_output=True, text=True)

    if ollama_installed.returncode == 0:
        print("Ollama installation was successful.")

        # Start an HTTP tunnel on the desired port, e.g., 11434
        tunnel = ngrok.connect(11434)
        print("Ngrok Tunnel URL:", tunnel.public_url)

        # Run Ollama serve
        ollama_process = subprocess.Popen(["ollama", "serve"])
        print("Ollama serve process started.")
    else:
        print("Ollama installation failed.")
        raise Exception("Ollama installation failed.")

except Exception as e:
    print("An error occurred:", str(e))
